In [167]:
import boto3
from botocore.exceptions import ClientError
import configparser

config = configparser.ConfigParser()
config.read('aws.cfg')

aws_access_key = config['AWS']['aws_access_key_id']
aws_secret_key = config['AWS']['aws_secret_access_key']


In [168]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1", aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

# Set the model ID, e.g., Titan Text Premier.
model_id = "amazon.titan-text-express-v1:0"

In [176]:
import pandas as pd
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

landmarks = s3.download_file('capstone-techcatalyst-conformed', 'group1/landmark_data/part-00000-tid-8270335314995015129-7c07741e-3c51-4b23-9a75-95c6c641450b-65-1-c000.csv', 'part-00000-tid-8270335314995015129-7c07741e-3c51-4b23-9a75-95c6c641450b-65-1-c000.csv')
landmark_df = pd.read_csv('/workspaces/thePANDAsCapstone/Pipeline Stages/Databricks/part-00000-tid-8270335314995015129-7c07741e-3c51-4b23-9a75-95c6c641450b-65-1-c000.csv')

In [198]:
landmark_list = landmark_df['Landmark_Name'].to_list()
description_list = landmark_df['USE_ORIG'].to_list()
build_type = landmark_df['BuildType'].to_list()

landmark_dict = {landmark_list[i]: description_list[i] for i in range(len(landmark_list))}

length = len(landmark_list)

landmark_str = ', '.join(landmark_list[:10])
#build_type_str = ', '.join(build_type)


print(length)
print('landmarks:', len(landmark_str))
print(landmark_str)
res


656
landmarks: 323
Brooklyn Bridge, Public School 15 (Daniel D. Tompkins School), Samuel Gompers Industrial High School, American Tract Society Building, Fire Engine Company No. 31, Steinway Hall, New York Public Library, Aguilar Branch, Guardian Life Insurance Company of America Annex, Hearst Magazine Building, Oliver Street Baptist Church


{'Brooklyn Bridge': 'Infrastructure, bridge',
 'Public School 15 (Daniel D. Tompkins School)': 'Civic, school',
 'Samuel Gompers Industrial High School': 'Civic, school',
 'American Tract Society Building': 'Commercial, office building',
 'Fire Engine Company No. 31': 'Civic, fire station',
 'Steinway Hall': 'Commercial, office building',
 'New York Public Library, Aguilar Branch': 'Civic, library',
 'Guardian Life Insurance Company of America Annex': 'Commercial, office building',
 'Hearst Magazine Building': 'Commercial, office building',
 'Oliver Street Baptist Church': 'Religious, church, Baptist',
 'Brooklyn Trust Company Building': 'Commercial, bank',
 'St. Regis Hotel': 'Commercial, hotel',
 'Westfield Township School No. 7': 'Civic, school',
 'Trinity Chapel Complex': 'Religious, church, Episcopal',
 'Public School 102': 'Civic, school',
 '193 Front Street Building': 'Commercial, countinghouse',
 'Church of Saint Vincent Ferrer Complex': 'Religious, church, Roman Catholic',
 'B

In [354]:
query2 = f""" {dict(list(landmark_dict.items())[10:21])}

Assign one of each one of these names to one of these categories using the description, choosing from: Architecture, Religion, Transportation, Education, Civic, Entertainment, Financial, Commercial, Industrial, Historic. Leave empty if none of the categories match. 

Use the following descriptions for the categories:
Architecture: infrastructure, structure
Religion: churches, religious
Transportation: airports, railroads, bridges
Education: school, university
Civic: government, military
Entertainment: arts, sports, theaters, museum, recreation, amusement
Financial: banks
Commercial: businesses, shopping, hotel
Industrial: factories
Historic: cemetery, relic


"""

print('length of string', len(query2))
print(query2)


length of string 1299
 {'Brooklyn Trust Company Building': 'Commercial, bank', 'St. Regis Hotel': 'Commercial, hotel', 'Westfield Township School No. 7': 'Civic, school', 'Trinity Chapel Complex': 'Religious, church, Episcopal', 'Public School 102': 'Civic, school', '193 Front Street Building': 'Commercial, countinghouse', 'Church of Saint Vincent Ferrer Complex': 'Religious, church, Roman Catholic', 'Benevolent and Protective Order of Elks, Lodge Number 878': 'Institutional, private club', 'Heckscher Building (now the Crown Building)': 'Commercial', 'Public School 86': 'Civic, school', 'Jackie Robinson Play Center': 'Civic, recreation, pool'}

Assign one of each one of these names to one of these categories using the description, choosing from: Architecture, Religion, Transportation, Education, Civic, Entertainment, Financial, Commercial, Industrial, Historic. Leave empty if none of the categories match. 

Use the following descriptions for the categories:
Architecture: infrastructure

In [355]:
# Start a conversation with the user message.
user_message = query2
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId="amazon.titan-text-express-v1",
        messages=conversation,
        inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
        additionalModelRequestFields={}
    )


    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)


except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Architecture:
Brooklyn Trust Company Building - Architecture
St. Regis Hotel - Architecture
Heckscher Building (now the Crown Building) - Architecture
Public School 102 - Architecture
193 Front Street Building - Architecture
Church of Saint Vincent Ferrer Complex - Architecture
Benevolent and Protective Order of Elks, Lodge Number 878 - Architecture
Public School 86 - Architecture

Religion:
Trinity Chapel Complex - Religion
Public School 102 - Religion
Church of Saint Vincent Ferrer Complex - Religion
Benevolent and Protective Order of Elks, Lodge Number 878 - Religion

Transportation:
None

Education:
Public School 102 - Education
Public School 86 - Education
Jackie Robinson Play Center - Education

Civic:
Westfield Township School No. 7 - Civic
Public School 102 - Civic
Public School 86 - Civic
Jackie Robinson Play Center - Civic

Entertainment:
None

Financial:
Brooklyn Trust Company Building - Financial
St. Regis Hotel - Financial
Heckscher Building (now the Crown Building) - Fina

In [371]:
#DO NOT RERUN
categories = {'Architecture':[], 'Transportation':[], 'Education':[], 'Religion':[], 'Civic':[], 'Entertainment':[], 'Religion':[], 'Civic':[], 'Entertainment':[], 'Financial':[], 'Commercial':[], 'Industrial':[], 'Historic':[]}


In [353]:

output = response_text.replace("\n\n", ":").split(':')

for i in range(1, len(output), 2):
    if output[i-1] in categories.keys():
        if output[i].strip() != 'None':
            if len(categories[output[i-1]]) != 0:
                categories[output[i-1]] = categories[output[i-1]] + [x.split('-')[0].strip() for x in output[i].strip().split('\n')]
            else:
                categories[output[i-1]] = output[i].strip().split('\n')
                categories[output[i-1]] = [x.split('-')[0].strip() for x in categories[output[i-1]]]
    else:
        continue

categories

{'Architecture': ['Brooklyn Bridge'],
 'Transportation': ['Brooklyn Bridge'],
 'Education': ['Public School 15 (Daniel D. Tompkins School)',
  'Samuel Gompers Industrial High School'],
 'Religion': ['Oliver Street Baptist Church'],
 'Civic': ['Fire Engine Company No. 31',
  'Guardian Life Insurance Company of America Annex',
  'Hearst Magazine Building',
  'New York Public Library, Aguilar Branch'],
 'Entertainment': [],
 'Financial': [],
 'Commercial': ['American Tract Society Building',
  'Brooklyn Trust Company Building'],
 'Industrial': [],
 'Historic': []}

In [323]:
def bedrock(query):
    user_message = query
    conversation = [
        {
            "role": "user",
            "content": [{"text": user_message}],
        }
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId="amazon.titan-text-express-v1",
            messages=conversation,
            inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
            additionalModelRequestFields={}
        )


        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

    return response_text

In [375]:
def categorize(start, end):
    print(f'rows {start} - {end}')
    query = f""" {dict(list(landmark_dict.items())[start:end])}

    Assign one of each one of these names to one of these categories using the description, choosing from: Architecture, Religion, Transportation, Education, Civic, Entertainment, Financial, Commercial, Industrial, Historic. Leave empty if none of the categories match. 

    Use the following descriptions for the categories:
    Architecture: infrastructure, structure
    Religion: churches, religious
    Transportation: airports, railroads, bridges
    Education: school, university
    Civic: government, military
    Entertainment: arts, sports, theaters, museum, recreation, amusement
    Financial: banks
    Commercial: businesses, shopping, hotel
    Industrial: factories
    Historic: cemetery, relic


    """

    response_text = bedrock(query)

    output = response_text.replace("\n", ":").split(':')

    for i in range(1, len(output), 2):
        if output[i-1].strip() in categories.keys():
            if output[i].strip() != 'None':
                if len(categories[output[i-1].strip()]) != 0:
                    categories[output[i-1].strip()] = categories[output[i-1].strip()] + [x.strip() for x in output[i].strip().split('\n')]
                else:
                    categories[output[i-1].strip()] = output[i].strip().split('\n')
                    categories[output[i-1].strip()] = [x.strip() for x in categories[output[i-1].strip()]]
        else:
            continue


In [376]:
for i in range(10, len(landmark_dict.keys()), 10):             
    categorize(i-10, i)

rows 0 - 10
Architecture: Brooklyn Bridge
    Religion: Oliver Street Baptist Church
    Transportation: Brooklyn Bridge
    Education: Public School 15 (Daniel D. Tompkins School)
    Civic: Samuel Gompers Industrial High School
    Entertainment: Fire Engine Company No. 31
    Financial: Guardian Life Insurance Company of America Annex
    Commercial: American Tract Society Building, Steinway Hall, Hearst Magazine Building, Fire Engine Company No. 31, Guardian Life Insurance Company of America Annex
    Industrial: American Tract Society Building
    Historic: Public School 15 (Daniel D. Tompkins School)
rows 10 - 20
Architecture:
    193 Front Street Building - Commercial, countinghouse
    Heckscher Building (now the Crown Building) - Commercial
    Brooklyn Trust Company Building - Commercial, bank
    Trinity Chapel Complex - Religious, church, Episcopal
    Public School 86 - Civic, school
    Public School 102 - Civic, school

     Religion:
    Church of Saint Vincent Ferrer C

UnboundLocalError: local variable 'response_text' referenced before assignment

In [377]:
categories

{'Architecture': ['Brooklyn Bridge',
  'Brooklyn Bridge',
  '',
  'Brooklyn Bridge',
  '',
  '',
  '',
  '',
  'Fort Totten Battery',
  'Bowery Bank of New York Building',
  'Marble Collegiate Church',
  'Fire Engine Company No. 252',
  'Engine Company No. 252',
  'Transportation',
  '',
  '',
  '',
  '',
  '',
  'Architecture',
  '',
  '',
  '',
  '',
  '',
  'Aeolian Building, Seagram Building, Longacre Theater, 144 West 14th Street Building',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'Flatiron Building, Puck Building',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'Gothic Hotel, Aschenbroedel Verein Building',
  '',
  '',
  'Coty Building, Union Building',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 'Transportation': ['Brooklyn Bridge',
  'Brooklyn Bridge',
  'Brooklyn Bridge',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'The Cyclone',
  '',
  '',
  '369th Regiment Armory',
  '',
  '',
  '',
  ''],
 'Educa

In [348]:
def categorize_place(description):

    query = f""" {description}

    Assign landmark name to one of these categories using the description, choosing from: Architecture, Religion, Transportation, Education, Civic, Entertainment, Financial, Commercial, Industrial, Historic. Return description and category.

    Use the following descriptions for the categories:
    Architecture: infrastructure, structure
    Religion: churches, religious
    Transportation: airports, railroads, bridges
    Education: school, university
    Civic: government, military
    Entertainment: arts, sports, theaters, museum, recreation, amusement
    Financial: banks
    Commercial: businesses, shopping, hotel
    Industrial: factories
    Historic: cemetery, relic


    """

    user_message = query
    conversation = [
        {
            "role": "user",
            "content": [{"text": user_message}],
        }
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId="amazon.titan-text-express-v1",
            messages=conversation,
            inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
            additionalModelRequestFields={}
        )


        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

    return response_text


In [349]:
categorize_place(dict(list(landmark_dict.items())[15:16]))


The description 193 Front Street Building is classified as Commercial.


'\nThe description 193 Front Street Building is classified as Commercial.'